# Defines

In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pickle
import sys
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch import optim
import torch.nn as nn
#from ipdb import set_trace
from datetime import datetime
import time
import copy
from tqdm import tqdm
from ipdb import set_trace
import argparse
from torch.utils.data import Dataset
from matplotlib import image
from matplotlib import pyplot
from torchvision import transforms
from PIL import Image
import imgaug as ia
import imgaug.augmenters as iaa

# Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

ROOT_DIR        = '../'
OLD_DATASET_DIR = ROOT_DIR + 'datasets/ntu_dataset/Training_data/'
NEW_DATASET_DIR = ROOT_DIR + 'datasets/ntu_dataset/Training_data_v2/'

# Util functions

In [2]:
def onehot(vals, possible_vals):
    if not isinstance(possible_vals, list): raise TypeError("provide possible_vals as a list")
    enc_vals = np.zeros([len(vals), len(possible_vals)])
    for i, value in enumerate(vals):
        if isinstance(possible_vals[0], float):
            enc = np.where(abs(possible_vals-value)<1e-3)
        else:
            enc = np.where(possible_vals==value)
        enc_vals[i,enc] = 1
    return enc_vals

def get_traffic_light_one_hot(traffic_light_state) : 
    if(traffic_light_state[0] == True) : 
#         return np.array([0,1])
        return 0
    else : 
        return 1
#         return np.array([1,0])

class Rescale(object):
    def __init__(self, scalar):
        self.scalar = scalar

    def __call__(self, im):
        w, h = [int(s*self.scalar) for s in im.size]
        return transforms.Resize((h, w))(im)

class Crop(object):
    def __init__(self, box):
        assert len(box) == 4
        self.box = box

    def __call__(self, im):
        return im.crop(self.box)

class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])

def load_checkpoint(filename) :
    #print(filename)
    checkpoint = torch.load(filename)
    model_state_dict = checkpoint['model_state_dict']
    optimizer_state_dict = checkpoint['optimizer_state_dict']
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return [model_state_dict, optimizer_state_dict, epoch, loss]

def save_checkpoint(model, optimizer, epoch, loss, filename):
    print("Saving Checkpoint....")
    model_state_dict = model.state_dict()
    optimizer_state_dict = optimizer.state_dict()
    torch.save({
            'epoch': epoch,
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'loss': loss
            }, filename)
    print("Saving Checkpoint Done")

def display_image(image) : 
    plt.show(image)
    
def display_torch_im(tensor) : 
    print(tensor.size())
    tensor_img_0 = tensor[0].squeeze(0)
    print(tensor_img_0.size())
    plt.imshow(tensor_img_0.permute(1, 2, 0))
    plt.show()
    
def load_checkpoint_weights(checkpoint, model) :
    print("checkpoint name : ", checkpoint)
    _model_state_dict, _optimizer_state_dict, _epoch, _loss = load_checkpoint(checkpoint)
    model.load_state_dict(_model_state_dict)

def resume_model(checkpoint, model, optimizer) : 
    print("checkpoint name : ", checkpoint)
    _model_state_dict, _optimizer_state_dict, _epoch, _loss = load_checkpoint(checkpoint)
    #print(_model_state_dict)
    model.load_state_dict(_model_state_dict)
    optimizer.load_state_dict(_optimizer_state_dict)
    print("resume model succesful")

def plot_tensor_image(tensor_im) :
    plt.imshow(tensor_im.permute(1, 2, 0))
    plt.show()



# Model functions

In [3]:
def train_model(model, dataloaders, dataset_sizes, criterion_dict, criterion_vec, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc       = 0.0
    best_loss      = 0.0
    epoch_acc      = 0.0
    num_outputs    = len(criterion_vec)
     
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, data in enumerate(dataloaders[phase]) :
                inputs_old     = data[0]
                labels         = data[1]
                inputs, labels = convert_inputs(inputs_old ,labels)
                inputs         = inputs.to(device)
                labels         = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                current_batch_size = inputs.size(0)
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    regression_preds, classification_preds = \
                              custom_preds(outputs, current_batch_size, num_outputs)
                    loss    = custom_loss(outputs, labels, criterion_dict, criterion_vec)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        if(i%100 == 0) :
                            print()
#                             print("preds shape : ", preds.size(), " labels shape : ", labels.size())
#                             print("preds : ", preds[0:5].T.data, "\nlabels : ", labels.data[0:5].T.data)
                            print_model_output("preds", regression_preds, classification_preds)
                            print_model_label("labels", labels)
#                             print_model_output("labels", regression_preds, classification_preds)
                            print("loss :", loss.item())

                # statistics
                running_loss += loss.item() * inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)

                if(phase == 'val') :
                    if(i%10==0) :
#                         print("preds : ", preds[0:5].T.data, "\nlabels : ", labels.data[0:5].T.data)
                        print_model_output("preds", regression_preds, classification_preds)
                        print_model_label("labels", labels)
                        print("loss :", loss.item())
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            #epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            #if phase == 'val' and epoch_acc > best_acc:
            if phase == 'val' and epoch_loss < best_loss:
                #best_acc = epoch_acc
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train'  :
                checkpoint_name = ROOT_DIR + 'checkpoints/model_' + str(epoch) + '.tar'
                save_checkpoint(model, optimizer, epoch, epoch_loss, checkpoint_name)
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def test_model(model, dataloaders, dataset_sizes, criterion_vec):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 0.0
    model.eval()   # Set model to evaluate mode
    running_corrects = 0
    running_loss = 0.0
    num_outputs    = len(criterion_vec)
    # Iterate over data.
    for i, data in enumerate(dataloaders) :
        inputs = data[0] 
        labels = data[1]
        inputs, labels = convert_inputs(inputs, labels)
        current_batch_size = inputs.size(0)
        if(i%10 == 0) :
            display_torch_im(inputs)
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
#         preds = outputs
        regression_preds, classification_preds = \
                              custom_preds(outputs, current_batch_size, num_outputs)
#         loss = criterion(outputs, labels)
        loss = custom_loss(outputs, labels)
        if(i%10 == 0) :
            print_model_output("preds", regression_preds, classification_preds, 1)
            print_model_label("labels", labels, 1)
            
        running_loss += loss.item() * inputs.size(0)
            
    time_elapsed = time.time() - since
    print('Test complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    return model

# Dataset

In [4]:
def get_augmentations():
    # applies the given augmenter in 50% of all cases,
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    # Define our sequence of augmentation steps that will be applied to every image
    seq = iaa.Sequential([
            # execute 0 to 5 of the following (less important) augmenters per image
            iaa.SomeOf((0, 5),
                [
                    iaa.OneOf([
                        iaa.GaussianBlur((0, 3.0)),
                        iaa.AverageBlur(k=(2, 7)),
                        iaa.MedianBlur(k=(3, 11)),
                    ]),
                    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
                    iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
                    # search either for all edges or for directed edges,
                    # blend the result with the original image using a blobby mask
                    iaa.SimplexNoiseAlpha(iaa.OneOf([
                        iaa.EdgeDetect(alpha=(0.5, 1.0)),
                        iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                    ])),
                    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                    iaa.OneOf([
                        iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                        iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                    ]),
                    iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                    iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                    # either change the brightness of the whole image (sometimes
                    # per channel) or change the brightness of subareas
                    iaa.OneOf([
                        iaa.Multiply((0.5, 1.5), per_channel=0.5),
                        iaa.FrequencyNoiseAlpha(
                            exponent=(-4, 0),
                            first=iaa.Multiply((0.5, 1.5), per_channel=True),
                            second=iaa.ContrastNormalization((0.5, 2.0))
                        )
                    ]),
                    iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                    sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                ],
                random_order=True
            )
        ],
        random_order=True
    )
    return seq
class Augment(object):
    def __init__(self, seq):
        self.seq = seq

    def __call__(self, im):
        return Image.fromarray(self.seq.augment_images([np.array(im)])[0])
    

class CARLA_Dataset(Dataset):
    def __init__(self, t, pickle_file_path_l, image_dir_path, pickle_file_path_r):
        #initializing transforms
        assert t in ['train', 'val']
        self.transform = get_data_transforms(t)
#         selabelslf.labels = {}    
        ###############LEFT CAMERA##############
        #reading the title of all images to access the pickle file
        self.image_path = image_dir_path
        self.image_list = os.listdir(image_dir_path)
        self.file_name = []
        for file in self.image_list:
            self.file_name.append(os.path.splitext(file)[0])
            
        #initialize the labels 
        #read pickle file
        self.pickle_list_l = os.listdir(pickle_file_path_l)
        self.pickled_data_l = {}
        for file in self.pickle_list_l:
            f = open((pickle_file_path_l + file), 'rb')  
            self.pickled_data_l.update(pickle.load(f))
            f.close() 
        
        ###############RIGHT CAMERA##############
        #read pickle file
        self.pickle_list_r = os.listdir(pickle_file_path_r)
        self.pickled_data_r = {}
        for file in self.pickle_list_r:
            f = open((pickle_file_path_r + file), 'rb')  
            self.pickled_data_r.update(pickle.load(f))
            f.close() 
        f.close() 

    def __len__(self):
        return len(self.file_name)

    def __getitem__(self, idx):
        frames      = []
        inputs = {}  
        labels = {}
        #####################LEFT CAMERA################
        #reading PIL
#         self.raw_image = Image.open(os.path.join(self.image_path, self.image_list[idx])).convert('RGB')
        raw_image = Image.open(os.path.join(self.image_path, self.image_list[idx])).convert('RGB')
        #pyplot.imshow(self.raw_image_l)
        #pyplot.show()
        
        #reading file name to access the pickle file
        current_fname = self.file_name[idx]
#         print(current_fname)
        chk_camera = current_fname[-3:]

        if chk_camera == '_rt' :
            current_fname_r = current_fname[:-3]
            label_dict = self.pickled_data_r[current_fname_r] 
        else :
            label_dict = self.pickled_data_l[current_fname] 
        label_values = list(label_dict.values())
#         print(label_values)
        #transforming regression labels
        labels['front_vehicle']       = torch.Tensor(np.array(float(label_dict['front_vehicle'])))
        labels['centre_dist']         = torch.Tensor(np.array(float(label_dict['centre_dist'])))
        labels['pedestrian_distance'] = torch.Tensor(np.array(float(label_dict['pedestrian_distance'])))
        labels_traffic_light          = get_traffic_light_one_hot(label_dict['traffic_light'])
        labels['traffic_light']       = torch.Tensor(np.array(float(labels_traffic_light)))
        labels['relative_angle'] = torch.Tensor(np.array(float(label_dict['relative_angle'])))
        im = self.transform(raw_image)
        
        inputs ['sequence'] = im
        return inputs, labels

#dataset support functions
def get_data_transforms(t='train'):
    data_transforms = {
        'train': transforms.Compose([
#             Augment(get_augmentations()),
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
        'val': transforms.Compose([
            Crop((0,120,800,480)),
            Rescale(0.4),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    return data_transforms[t]

def get_data(batch_size):
    dataset_dir = NEW_DATASET_DIR
#     dataset_dir = OLD_DATASET_DIR
    train_ds = CARLA_Dataset( 'train', pickle_file_path_l = dataset_dir+'/Left_pickle_file/',
                                       image_dir_path     = dataset_dir+'/Camera/', 
                                       pickle_file_path_r = dataset_dir+'/Right_pickle_file/')
    val_ds   = CARLA_Dataset( 'val',   pickle_file_path_l = dataset_dir+'/Left_pickle_file/',
                                       image_dir_path     = dataset_dir+'/Camera/', 
                                       pickle_file_path_r = dataset_dir+'/Right_pickle_file/')
#     return (len(train_ds),len(val_ds),DataLoader(train_ds, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=10),
#         DataLoader(val_ds, batch_size=batch_size*2, pin_memory=True, num_workers=10),
#     )
    return (
        len(train_ds),
        len(val_ds),
        DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=1),
        DataLoader(val_ds, batch_size=batch_size*2, num_workers=1),
        train_ds,
        val_ds
    )


In [5]:
len_train_ds, len_valid_ds, train_dl, valid_dl, train_ds, val_ds = get_data(batch_size=64)
print(len_train_ds)
print(len_valid_ds)

dataloaders   = {'train':train_dl, 'val':valid_dl}
print(dataloaders['train'])
dataset_sizes = {'train':len_train_ds, 'val':len_valid_ds}
print("dataset sizes :", dataset_sizes)

42928
42928
dataset sizes : {'val': 42928, 'train': 42928}


# Model Definitions

## Model for combined for regression and classification prediction

In [6]:
def custom_preds(output, batch_size, num_outputs) : 
#     print("custom preds : ", output.size(), batch_size, num_outputs)
    num_pred_outputs = num_outputs-1
    num_regression_outputs = 3
    
    regression_preds     = torch.zeros(batch_size, num_regression_outputs)
    classification_preds = torch.zeros(batch_size) 
    
    regression_preds        = output[:,0:3]
    _, classification_preds = torch.max(output[:,3:5], 1) #.float()
    classification_preds    = classification_preds.T.float()
    classification_preds    = classification_preds.unsqueeze(dim=1)
#     print("outputs : ", output)
#     print("classificaton out : ", output[:,2:4])
#     print("regression output : ", regression_preds)
#     print("classification output : ", classification_preds)
    return regression_preds, classification_preds
    
def custom_loss(output, target, criterion_dict=None, criterion_vec=None) : 
    regression_loss     = 0.0
    classification_loss = 0.0
    loss     = 0.0
    criterion_mse       = nn.MSELoss()
    criterion_bce_logit = nn.BCEWithLogitsLoss()
    criterion_ce        = nn.CrossEntropyLoss()
    for i in range(3) : 
        regression_loss     += criterion_mse(output[:,i], target[:,i])
    classification_target = target[:,3].long()
    classification_target = classification_target #.squeeze(dim=1)
    classification_output = output[:,3:5]
    classification_loss = criterion_ce(classification_output, classification_target)
    loss = regression_loss + classification_loss

    return loss

#model definitions (combined regression)
def convert_inputs(inputs_old, labels):
    inputs               = inputs_old['sequence']
    labels_vdistance     = labels['front_vehicle']/50
    labels_cdistance     = labels['centre_dist']
    labels_direction     = labels['pedestrian_distance']
    labels_rangle        = labels['relative_angle']
    labels_traffic_light = labels['traffic_light']
    labels_traffic_light = labels_traffic_light#.squeeze(dim=1)
    combined_label       = torch.stack([labels_vdistance, labels_cdistance, labels_rangle, labels_traffic_light], dim=1)
    return inputs, combined_label

def print_model_output(name, regression_out, classification_out, num_output=5) : 
    print("****** ", name, " ******")
    print("VD : ", regression_out[0:num_output,0].cpu().detach().numpy(), \
          " \nCD : ", regression_out[0:num_output,1].cpu().detach().numpy(), \
          " \nRA : ", regression_out[0:num_output,2].cpu().detach().numpy(), \
          " \nTL : ", classification_out[0:num_output].T.cpu().detach().numpy())
    
def print_model_label(name, label, num_output=5) :
    print("****** ", name, " ******")
    print("VD : ", label[0:num_output,0].cpu().detach().numpy(), \
          " \nCD : ", label[0:num_output,1].cpu().detach().numpy(), \
          " \nRA : ", label[0:num_output,2].cpu().detach().numpy(), \
          " \nTL : ", label[0:num_output,3].cpu().detach().numpy())

device           = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft         = models.resnet50(pretrained=True)
criterion_vec    = [1,1,0,0]
optimizer_ft     = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
num_ftrs         = model_ft.fc.in_features
model_ft.fc      = nn.Linear(num_ftrs, 5)
model_ft         = nn.DataParallel(model_ft)
model_ft         = model_ft.to(device)
# load_checkpoint_weights(ROOT_DIR +'checkpoints/vehicle_distance/model_14.tar', model_ft)
# model_ft.fc      = nn.Linear(num_ftrs, 4)
# model_ft         = nn.DataParallel(model_ft)
# model_ft         = model_ft.to(device)
# optimizer_ft     = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
# print(model_ft.fc)
# print(model_ft.fc.weight.size())
# print(model_ft.fc.bias)
# print("copying model to device....", device)

# Training

In [7]:
# # #train model
# print("starting training...")
# # #if(args.retrain == True) : 
# #  #   resume_model(args.checkpoint, model_ft, optimizer_ft)
resume_model(ROOT_DIR + 'checkpoints/model_14.tar', model_ft, optimizer_ft) 
criterion_dict     = {'MSE' : nn.MSELoss, 'CE' : nn.CrossEntropyLoss}
train_model(model_ft, dataloaders, dataset_sizes, criterion_dict, criterion_vec, optimizer_ft, exp_lr_scheduler, 50)

checkpoint name :  ../checkpoints/model_14.tar
resume model succesful
Epoch 0/49
----------

******  preds  ******
VD :  [1.0176787  0.90992504 0.9954159  0.99996924 1.0093995 ]  
CD :  [0.187402   0.8275923  0.25683743 0.42294487 0.38526636]  
RA :  [-6.278695   -6.1687837   0.09608513 -6.1670027  -0.06012247]  
TL :  [[1. 1. 0. 1. 1.]]
******  labels  ******
VD :  [1. 1. 1. 1. 1.]  
CD :  [0.14545526 0.92734075 0.28051636 0.46357667 0.48070443]  
RA :  [-6.2837844e+00 -6.1782041e+00  2.5286656e-04 -6.2821221e+00
  5.4714642e-04]  
TL :  [1. 1. 0. 1. 1.]
loss : 0.18914327025413513

******  preds  ******
VD :  [1.0110155  0.96326196 0.21097897 0.98529565 0.31719208]  
CD :  [0.24882221 0.62423235 0.42392805 0.28584865 0.5310285 ]  
RA :  [-2.4644937e-03 -6.1878543e+00  3.4727994e-03 -8.6079665e-02
  5.3945102e-02]  
TL :  [[1. 1. 1. 0. 1.]]
******  labels  ******
VD :  [1.         1.         0.17670164 1.         0.25662297]  
CD :  [0.24142064 0.59120786 0.4792377  0.28051636 0.505705

******  preds  ******
VD :  [1.0415105  1.0279802  0.2719289  0.9673481  0.97338784]  
CD :  [0.39150456 0.4176234  0.2105396  0.3807505  0.10107037]  
RA :  [ 0.01080209 -0.09177734  0.01392177 -6.133259   -6.250334  ]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [1.         1.         0.2286734  1.         0.98009825]  
CD :  [0.48064375 0.47982147 0.22257237 0.22392902 0.12108105]  
RA :  [-1.3446300e-03 -6.3383236e-04  6.3063652e-04 -6.3125691e+00
 -6.2826638e+00]  
TL :  [1. 1. 1. 1. 1.]
loss : 0.16707569360733032
******  preds  ******
VD :  [0.28873467 1.1310571  1.044784   0.20557582 0.17512937]  
CD :  [0.22032051 0.3001017  0.34814814 0.33582234 0.3221767 ]  
RA :  [-0.0496974   6.068257   -0.04728434 -0.04664224  0.00609895]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [0.21787253 1.         1.         0.18692186 0.18462671]  
CD :  [0.21766956 0.32539427 0.3279805  0.33854002 0.33304733]  
RA :  [-5.9228705e-04  6.2830348e+00  4.9082062e-04 -6.6246645e-

******  preds  ******
VD :  [0.14512272 1.0328554  0.19802652 0.92513806 0.1338742 ]  
CD :  [0.18539165 0.50454074 0.47811538 0.4627245  0.61656755]  
RA :  [-4.7475826e-03 -6.1822190e+00 -5.2678026e-02  2.2384861e-02
 -2.4373289e-03]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [0.14059614 1.         0.19736224 0.89662725 0.14203332]  
CD :  [0.1707548  0.5128024  0.47130808 0.48290867 0.5827151 ]  
RA :  [-1.0833739e-03 -6.2831373e+00 -4.7537425e-04  1.9654128e-04
 -9.6819730e-04]  
TL :  [1. 1. 1. 1. 1.]
loss : 0.08438781648874283
******  preds  ******
VD :  [0.9914496  0.9997543  0.20146126 0.25408337 0.5347413 ]  
CD :  [0.2848877  0.26891384 0.23596525 0.3622019  0.50909173]  
RA :  [-4.4979587e-02  3.1420030e-02  6.3117490e+00 -3.8672406e-03
  4.1457392e-02]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [1.         1.         0.13815306 0.21265808 0.5920582 ]  
CD :  [0.2706801  0.25354588 0.2655627  0.32380578 0.47677281]  
RA :  [ 8.0226511e-03 -4.1438788

train Loss: 0.3156 Acc: 0.0000
Saving Checkpoint....
Saving Checkpoint Done
******  preds  ******
VD :  [0.94977534 0.7813115  1.0383528  0.20847352 0.95893013]  
CD :  [0.4542672  0.5169729  0.41117236 0.07977961 0.47262418]  
RA :  [ 0.02207173 -0.05010989  0.13064922 -0.0105176  -6.0075145 ]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [1.        0.7703713 1.        0.2435772 1.       ]  
CD :  [0.46922055 0.4811718  0.47055134 0.02217751 0.49978843]  
RA :  [ 7.5374648e-02 -4.3489421e-03  5.6352490e-04 -2.5006838e-04
 -6.2826858e+00]  
TL :  [1. 1. 1. 1. 1.]
loss : 0.23515473306179047
******  preds  ******
VD :  [0.4125539  0.98296106 0.14083768 0.24513717 0.55379534]  
CD :  [0.33242756 0.5039795  0.18785223 0.7793284  0.5910329 ]  
RA :  [-0.12995048  6.444309   -0.01146924  0.06451649 -0.02473191]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [0.46792457 1.         0.14059614 0.21123414 0.60045743]  
CD :  [0.2236872  0.4974619  0.1707548  0.7026558  0.71874

******  preds  ******
VD :  [0.90705484 0.22876601 0.9952228  1.0144278  0.49378532]  
CD :  [0.42355847 0.6070966  0.37167174 0.39282575 0.24740177]  
RA :  [ 0.05865911  0.11099813 -0.23705706  0.06939315  0.05026943]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [1.         0.23499368 1.         1.         0.5044433 ]  
CD :  [0.35415572 0.6616288  0.31698748 0.3357934  0.22585769]  
RA :  [ 0.07080546  0.10382929  0.00018882  0.00046525 -0.00129536]  
TL :  [1. 1. 1. 1. 1.]
loss : 0.2727280557155609
******  preds  ******
VD :  [0.16154979 0.9283134  0.95895547 0.12335906 0.36222076]  
CD :  [0.62087864 0.21771729 0.6038373  0.3395037  0.5080872 ]  
RA :  [ 0.01849481  0.00910068 -0.07581362  7.029246   -0.02351228]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [0.13926408 0.8694768  1.         0.21174766 0.14875618]  
CD :  [0.58247447 0.23469344 0.52582085 0.26316145 0.52241546]  
RA :  [-3.0186930e-04  1.7829863e-03 -3.0399984e-04  6.2824969e+00
 -1.6835019e-0

...

******  preds  ******
VD :  [0.88408715 0.90997165 0.95707595 0.23373328 0.24612589]  
CD :  [0.26108032 0.17508808 0.4194344  0.5367829  0.19003975]  
RA :  [ 0.02136021  0.2552911  -0.07973977 -0.00531952  0.32495144]  
TL :  [[0. 1. 1. 1. 1.]]
******  labels  ******
VD :  [0.89895684 0.82978094 1.         0.28496024 0.21432804]  
CD :  [0.28302887 0.08553156 0.3145771  0.36211672 0.21798423]  
RA :  [ 0.00025074 -0.00073557 -0.11342856 -0.12401116  0.00357183]  
TL :  [0. 1. 1. 1. 1.]
loss : 0.013708563521504402
******  preds  ******
VD :  [0.97498333 0.11918468 0.17940702 1.0172021  0.99222815]  
CD :  [0.52306217 0.00194074 0.06653142 0.20974472 0.09494299]  
RA :  [-6.371275    6.225444    0.06899648  6.38347    -0.02829721]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [1.         0.13508287 0.23409753 1.         1.        ]  
CD :  [0.73658353 0.01589845 0.01139818 0.32477432 0.07064857]  
RA :  [-6.3478074e+00  6.2826672e+00  9.1501170e-06  6.2830410e+00
  5.1652011e


******  preds  ******
VD :  [0.2341563  0.98866117 0.12846464 0.18134168 0.69094765]  
CD :  [0.41148186 0.5539306  0.2187775  0.54776686 0.237692  ]  
RA :  [-6.2518096e+00 -6.3893390e+00 -2.2725938e-02 -1.2450051e-03
 -1.1590675e-01]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [0.24376495 1.         0.15723945 0.21752557 0.55509585]  
CD :  [0.4346143  0.52982587 0.25323167 0.57304364 0.2575982 ]  
RA :  [-6.2830887e+00 -6.3039851e+00  6.2291336e-04 -6.8004330e-04
 -7.3197519e-04]  
TL :  [1. 1. 1. 1. 1.]
loss : 0.0116128446534276

******  preds  ******
VD :  [0.98334324 0.23752068 0.9986824  0.15865014 0.22157983]  
CD :  [0.23728013 0.595916   0.4584991  0.2149995  0.01287822]  
RA :  [ 1.4364297e-02  6.1118282e-02  8.2776569e-02 -6.2920365e+00
 -6.2035304e-03]  
TL :  [[0. 1. 1. 1. 1.]]
******  labels  ******
VD :  [1.         0.23986557 1.         0.1379526  0.23601805]  
CD :  [0.25287578 0.5788688  0.50159293 0.26318833 0.01269012]  
RA :  [ 1.9707392e-04 -6.241169

******  preds  ******
VD :  [1.0457007  0.62049776 1.0217488  0.96530294 0.28400055]  
CD :  [0.5062377  0.42964202 0.42093545 0.5600307  0.38437554]  
RA :  [-0.06898654 -0.00129448  0.06318333  0.03109241 -0.12001147]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [1.         0.3899584  1.         1.         0.18474431]  
CD :  [0.54431593 0.24447314 0.502681   0.5456907  0.41942716]  
RA :  [-0.0005827   0.0009547   0.00092434 -0.0003835  -0.00156061]  
TL :  [1. 1. 1. 1. 1.]
loss : 0.012578334659337997
******  preds  ******
VD :  [0.42744943 0.988193   0.98027706 0.25659811 0.22727376]  
CD :  [0.63703483 0.4787571  0.37774944 0.25673294 0.33931157]  
RA :  [-5.7281412e-02 -3.4889124e-02  6.1830668e+00 -2.2852849e-03
 -5.3150639e-02]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [0.2438306  1.         1.         0.223041   0.24282973]  
CD :  [0.45646423 0.52901757 0.23182984 0.279695   0.27266884]  
RA :  [-6.1785337e-05  1.1187246e-03  6.2822661e+00 -3.1558215e

******  preds  ******
VD :  [1.0130655  0.73251    0.97104704 0.1687496  1.0162486 ]  
CD :  [0.6849875  0.33894944 0.5491471  0.77938336 0.47333953]  
RA :  [-6.341305   -6.7204795   0.1988914  -0.06361419  0.07151666]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [1.         0.8301579  1.         0.21822193 1.        ]  
CD :  [0.7352597  0.33685    0.46806183 0.7222007  0.52243364]  
RA :  [-6.4113321e+00 -6.2833676e+00 -3.7843519e-04  6.8336714e-04
  3.7392911e-02]  
TL :  [1. 1. 1. 1. 1.]
loss : 0.017121925950050354
******  preds  ******
VD :  [0.13445579 0.80589926 0.9954324  0.9554938  0.21875402]  
CD :  [0.42774    0.40145415 0.37035906 0.601168   0.74656636]  
RA :  [ 3.6998782e-03 -6.5979896e+00 -6.2891526e+00 -2.8527224e-02
 -2.8414851e-02]  
TL :  [[1. 1. 1. 1. 1.]]
******  labels  ******
VD :  [0.14059614 0.8036387  1.0015193  0.9983322  0.24401201]  
CD :  [0.42075482 0.39835504 0.3697016  0.5725264  0.73336107]  
RA :  [-1.0833739e-03 -6.2833729e+00 -6.2826161

KeyboardInterrupt: 

# Testing Model

In [ ]:
load_checkpoint_weights(ROOT_DIR +'checkpoints/combined_ntu_data_vd_cd_tl/model_14.tar', model_ft)
test_model(model_ft, dataloaders['val'], dataset_sizes['val'], criterion_vec)

# Dataloader unit test

In [ ]:
# #model definitions (combined regression)
# def convert_inputs(inputs_old, labels):
#     inputs               = inputs_old['sequence']
#     labels_vdistance     = labels['front_vehicle']/50
#     labels_cdistance     = labels['centre_dist']
#     labels_direction     = labels['pedestrian_distance']
#     labels_traffic_light = labels['traffic_light']
# #     labels_traffic_light = labels_traffic_light.T
#     #print("driection shape : ", labels_direction.size())
#     print("labels traffic before : ", labels_traffic_light.size())
#     labels_traffic_light = labels_traffic_light.squeeze(dim=1)
#     print("labels traffic after : ", labels_traffic_light.size())
# #     combined_label       = torch.stack([labels_vdistance, labels_cdistance, labels_traffic_light], dim=1)
#     combined_label       = torch.stack([labels_vdistance, labels_cdistance], dim=1)
    
#     print(combined_label)
#     return inputs, combined_label

# train_dataloader                 = dataloaders['train']
# data                             = next(iter(train_dataloader))
# test_input , test_train_label    = data
# converted_input, converted_label = convert_inputs(test_input, test_train_label)

# test_train_im = test_input['sequence']
# print(test_train_label)
# print(test_train_im.size())
# for i in range(16) :
#     print(test_train_label['front_vehicle'][i])
#     print("converted label : ", converted_label[i])
#     plot_tensor_image(test_train_im[i])